<center>
        <div style="font-size: 40px; color: black"><b>Extraction of 3D information from binocular images</b></div>
</center>

EPPS Training School 12.09.2025

Samuel Thomas - Arvalis (s.thomas@arvalis.fr)

license under CC By NC 4.0

# Part 1 : Camera calibration

In this notebook, you will find the code for the 2 steps required before thinking retrieving the 3D information from a stereo camera pair :

* Calibration of a single camera

* Calibration of the stereo pair formed by two calibrated cameras

Documentation may be found here :

https://docs.opencv.org/4.x/d6/d55/tutorial_table_of_content_calib3d.html

https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html

https://nikolasent.github.io/computervision/opencv/calibration/2024/12/20/Practical-OpenCV-Refinement-Techniques.html

In [29]:
import glob as glob
import os

import numpy as np

import calibrationlib

## We prepare the data, paths, and variables

In [30]:
# This calibration operation has to be done once theoritically, if the cameras are securely fixed together.
# A minimal set of ~10 images is required officially ; in practice we take around 50 images, trying to cover 
# the corners of the image, different orientations of the chessboard and a range of distances to the sensor
# consistent with our applications.
# The more images are well-exposed (taken outside for example), the less noise you will have and the better
# the chessboard detection will work

path_calibration = (r"./data/calibration/classical_system")

# Calibration paths for left ('Camera1') and right ('Camera2') chessboard images
path_images = [
    path_calibration + "/images/Left",
    path_calibration + "/images/Right",
]

path_matrices = path_calibration + "/example_matrices"
if os.path.isdir(path_matrices) is False:
    os.mkdir(path_matrices)

path_matrices_single_cam = [
    path_matrices + "/Left",
    path_matrices + "/Right",
]

if os.path.isdir(path_matrices_single_cam[0]) is False:
    os.mkdir(path_matrices_single_cam[0])

if os.path.isdir(path_matrices_single_cam[1]) is False:
    os.mkdir(path_matrices_single_cam[1])

image_type = "jpg"

# Chessboard pattern size (width, height)
chessboard_pattern = (13, 8)
# Chessboard pattern cell size (here in mm) : used for scaling calibration parameters
chessboard_cell_size = 40

calibration = calibrationlib.CameraCalibration(chessboard_pattern, chessboard_cell_size)

count = 0

object_points_cam1 = []
image_points_cam1 = []
object_points_cam2 = []
image_points_cam2 = []

success_images_cam1 = []
success_images_cam2 = []

name = path_calibration + "/errors.log"
log = open(name, "w")

## First step - Part 1 : Calibration of the first (left) camera

In [31]:
# Computation of the calibration parameters for both left and right cameras (mandatory), then for the stereo camera pair

# Left/first camera
log_line = "Calibration - Left/first camera\r"
log.write(log_line)
print(log_line)

ret1 = False

(
    ret1,
    reprojection_error_cam1,
    success_images_cam1,
    roi_cam1,
    camera_matrix_cam1,
    dist_coefs_cam1,
    r_vecs_cam1,
    t_vecs_cam1,
    object_points_cam1,
    image_points_cam1,
    image_size_cam1,
) = calibration.single_calibration(
    path_images[0],
    image_type,
    log_file=log,
)

tmp = np.array(object_points_cam1)
name = path_matrices_single_cam[0] + "/object_points"
np.save(name, tmp)
tmp = np.array(image_points_cam1)
name = path_matrices_single_cam[0] + "/image_points"
np.save(name, tmp)
tmp = np.array(success_images_cam1)
name = path_matrices_single_cam[0] + "/success_images"
np.save(name, tmp)

tmp = np.array(image_size_cam1)
name = path_matrices_single_cam[0] + "/image_size"
np.save(name, tmp)

tmp = np.array(camera_matrix_cam1)
name = path_matrices_single_cam[0] + "/camera_matrix"
np.save(name, tmp)
tmp = np.array(dist_coefs_cam1)
name = path_matrices_single_cam[0] + "/dist_coefs"
np.save(name, tmp)
tmp = np.array(r_vecs_cam1)
name = path_matrices_single_cam[0] + "/r_vecs"
np.save(name, tmp)
tmp = np.array(t_vecs_cam1)
name = path_matrices_single_cam[0] + "/t_vecs"
np.save(name, tmp)
tmp = np.array(reprojection_error_cam1)
name = path_matrices_single_cam[0] + "/reprojection_error"
np.save(name, tmp)

log_line = f"End of left/first camera calibration successful : {ret1}\r"
log.write(log_line)
print(log_line)

Calibration - Left/first camera
10 images found
Entering image ./data/calibration/classical_system/images/Left/Camera1_1.jpg
Entering image ./data/calibration/classical_system/images/Left/Camera1_10.jpg
Entering image ./data/calibration/classical_system/images/Left/Camera1_2.jpg
Entering image ./data/calibration/classical_system/images/Left/Camera1_3.jpg
Entering image ./data/calibration/classical_system/images/Left/Camera1_4.jpg
Entering image ./data/calibration/classical_system/images/Left/Camera1_5.jpg
Entering image ./data/calibration/classical_system/images/Left/Camera1_6.jpg
Entering image ./data/calibration/classical_system/images/Left/Camera1_7.jpg
Entering image ./data/calibration/classical_system/images/Left/Camera1_8.jpg
Entering image ./data/calibration/classical_system/images/Left/Camera1_9.jpg
Mean error =  0.5490302814636513
Headcount =  10
Reprojection error =  0.05490302814636513
Rejected images :  []
End of left/first camera calibration successful : True


### Outputs :

Left camera matrix (3x3) :

In [32]:
camera_matrix_cam1

array([[3.39649371e+03, 0.00000000e+00, 2.39394645e+03],
       [0.00000000e+00, 3.39717247e+03, 1.62465482e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

Left distorsion coefficients vector (5x1) :

In [33]:
dist_coefs_cam1

array([[ 0.00619622],
       [ 0.04726172],
       [ 0.00143935],
       [-0.00130746],
       [-0.08854702]])

## First step - Part 2 : Calibration of the second (right) camera

In [34]:
# Right/second camera
log_line = "Calibration - Right/second camera\r"
log.write(log_line)
print(log_line)

ret2 = False

(
    ret2,
    reprojection_error_cam2,
    success_images_cam2,
    roi_cam2,
    camera_matrix_cam2,
    dist_coefs_cam2,
    r_vecs_cam2,
    t_vecs_cam2,
    object_points_cam2,
    image_points_cam2,
    img_size_cam2,
) = calibration.single_calibration(
    path_images[1],
    image_type,
    log_file=log,
)

tmp = np.array(object_points_cam2)
name = path_matrices_single_cam[1] + "/object_points"
np.save(name, tmp)
tmp = np.array(image_points_cam2)
name = path_matrices_single_cam[1] + "/image_points"
np.save(name, tmp)
tmp = np.array(success_images_cam2)
name = path_matrices_single_cam[1] + "/success_images"
np.save(name, tmp)

tmp = np.array(image_size_cam1)
name = path_matrices_single_cam[1] + "/image_size"
np.save(name, tmp)

tmp = np.array(camera_matrix_cam2)
name = path_matrices_single_cam[1] + "/camera_matrix"
np.save(name, tmp)
tmp = np.array(dist_coefs_cam2)
name = path_matrices_single_cam[1] + "/dist_coefs"
np.save(name, tmp)
tmp = np.array(r_vecs_cam2)
name = path_matrices_single_cam[1] + "/r_vecs"
np.save(name, tmp)
tmp = np.array(t_vecs_cam2)
name = path_matrices_single_cam[1] + "/t_vecs"
np.save(name, tmp)
tmp = np.array(reprojection_error_cam2)
name = path_matrices_single_cam[1] + "/reprojection_error"
np.save(name, tmp)

log_line = f"End of right/second camera calibration successfully achieved : {ret2}\r"
log.write(log_line)
print(log_line)

Calibration - Right/second camera
10 images found
Entering image ./data/calibration/classical_system/images/Right/Camera2_1.jpg
Entering image ./data/calibration/classical_system/images/Right/Camera2_10.jpg
Entering image ./data/calibration/classical_system/images/Right/Camera2_2.jpg
Entering image ./data/calibration/classical_system/images/Right/Camera2_3.jpg
Entering image ./data/calibration/classical_system/images/Right/Camera2_4.jpg
Entering image ./data/calibration/classical_system/images/Right/Camera2_5.jpg
Entering image ./data/calibration/classical_system/images/Right/Camera2_6.jpg
Entering image ./data/calibration/classical_system/images/Right/Camera2_7.jpg
Entering image ./data/calibration/classical_system/images/Right/Camera2_8.jpg
Entering image ./data/calibration/classical_system/images/Right/Camera2_9.jpg
Mean error =  0.5465064835691831
Headcount =  10
Reprojection error =  0.05465064835691831
Rejected images :  []
End of right/second camera calibration successfully achi

### Outputs :

Right camera matrix (3x3) :

In [35]:
camera_matrix_cam2

array([[3.38931531e+03, 0.00000000e+00, 2.42576843e+03],
       [0.00000000e+00, 3.38983502e+03, 1.62734145e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

Right distorsion coefficients vector (5x1) :

In [36]:
dist_coefs_cam2

array([[ 0.01312779],
       [ 0.01608511],
       [ 0.00112811],
       [-0.00090645],
       [-0.04848754]])

## Second step : Calibration of the stereo pair

In [37]:
# Calibration of the stereo pair
log_line = "Stereo calibration between left/first and right/second cameras\r"
log.write(log_line)
print(log_line)

if (ret1 is True) and (ret2 is True):

    # We keep only image pairs on which were successfully detected the chessboard corners
    # for both cameras, and store relative objects and image points for further stereo
    # calibration
    common_images, index_points_cam1, index_points_cam2 = np.intersect1d(success_images_cam1, success_images_cam2, return_indices=True)

    log_line = f"Common images : {(np.array(success_images_cam1))[index_points_cam1]}\r"
    log.write(log_line)
    log_line = f"Images indexes for camera 1 : {list(index_points_cam1)}\r"
    log.write(log_line)
    print(log_line)
    log_line = f"Images indexes for camera 2 : {list(index_points_cam2)}\r"
    log.write(log_line)
    print(log_line)


    # Resulting list of common points for the two cameras (i.e. from the image pairs on which
    # the chessboard has been successfully deteted in both cameras
    filtered_object_points_cam1 = np.array(object_points_cam1)
    filtered_object_points_cam1 = filtered_object_points_cam1[index_points_cam1]
    filtered_object_points_cam1 = list(filtered_object_points_cam1)
    
    filtered_image_points_cam1 = np.array(image_points_cam1)
    filtered_image_points_cam1 = filtered_image_points_cam1[index_points_cam1]
    filtered_image_points_cam1 = list(filtered_image_points_cam1)
    
    filtered_image_points_cam2 = np.array(image_points_cam2)
    filtered_image_points_cam2 = filtered_image_points_cam2[index_points_cam2]
    filtered_image_points_cam2 = list(filtered_image_points_cam2)

    # We compute now stereo calibration : camera_matrices and distorsion coefficients for each
    # camera may be slightly modified (depending on the flags parameters : in our case, the output 
    # values are strictly the same as the input ones), and we get the rotation matrix R and 
    # translation matrix T that bring points in the left camera coordinate system to points in the 
    # right camera coordinate system (you may read the doc : documentation/OpenCV - stereoCalibrate) 
    (
        ret,
        camera_matrix_cam1_stereo,
        dist_coefs_cam1_stereo,
        camera_matrix_cam2_stereo,
        dist_coefs_cam2_stereo,
        R,
        T,
    ) = calibration.stereo_calibration(
        filtered_object_points_cam1,
        filtered_image_points_cam1,
        filtered_image_points_cam2,
        camera_matrix_cam1,
        dist_coefs_cam1,
        camera_matrix_cam2,
        dist_coefs_cam2,
        image_size_cam1,
        log_file=log,
    )

    # We save the useful parameters, from which further rectification of images will 
    # be possible for matching. The rectification maps will be loaded once at the 
    # beginning of the matching code, and applied to each images pair.
    tmp = np.array(camera_matrix_cam1_stereo)
    name = path_matrices_single_cam[0] + "/camera_matrix_from_stereo"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam1_stereo)
    name = path_matrices_single_cam[0] + "/dist_coefs_from_stereo"
    np.save(name, tmp)
    tmp = np.array(camera_matrix_cam2_stereo)
    name = path_matrices_single_cam[1] + "/camera_matrix_from_stereo"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam2_stereo)
    name = path_matrices_single_cam[1] + "/dist_coefs_from_stereo"
    np.save(name, tmp)
    tmp = np.array(R)
    name = path_calibration + "/R"
    np.save(name, tmp)
    tmp = np.array(T)
    name = path_calibration + "/T"
    np.save(name, tmp)
    tmp = np.array(ret)
    name = path_calibration + "/reprojection_error"
    np.save(name, tmp)

    log_line = "End of stereo calibration successfully achieved\r"
    log.write(log_line)
    print(log_line)
    
else:
    log_line = "Problem with the calibration of one of the two cameras - no way to perform stereo calibration"
    log.write(log_line)
    print(log_line)

log_line = "End of calibration process"
log.write(log_line)
print(log_line)

Stereo calibration between left/first and right/second cameras
Images indexes for camera 1 : [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)]
Images indexes for camera 2 : [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)]
Reprojection error of the stereo pair =  1.4012701452391074
End of stereo calibration successfully achieved
End of calibration process


### Outputs :

In [38]:
camera_matrix_cam1

array([[3.39649371e+03, 0.00000000e+00, 2.39394645e+03],
       [0.00000000e+00, 3.39717247e+03, 1.62465482e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [39]:
camera_matrix_cam1_stereo

array([[3.39649371e+03, 0.00000000e+00, 2.39394645e+03],
       [0.00000000e+00, 3.39717247e+03, 1.62465482e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [40]:
dist_coefs_cam1

array([[ 0.00619622],
       [ 0.04726172],
       [ 0.00143935],
       [-0.00130746],
       [-0.08854702]])

In [41]:
dist_coefs_cam1_stereo

array([[ 0.00619622],
       [ 0.04726172],
       [ 0.00143935],
       [-0.00130746],
       [-0.08854702]])

In [42]:
camera_matrix_cam2

array([[3.38931531e+03, 0.00000000e+00, 2.42576843e+03],
       [0.00000000e+00, 3.38983502e+03, 1.62734145e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [43]:
camera_matrix_cam2_stereo

array([[3.38931531e+03, 0.00000000e+00, 2.42576843e+03],
       [0.00000000e+00, 3.38983502e+03, 1.62734145e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [44]:
dist_coefs_cam2

array([[ 0.01312779],
       [ 0.01608511],
       [ 0.00112811],
       [-0.00090645],
       [-0.04848754]])

In [45]:
dist_coefs_cam2_stereo

array([[ 0.01312779],
       [ 0.01608511],
       [ 0.00112811],
       [-0.00090645],
       [-0.04848754]])

In [46]:
R

array([[ 9.99987824e-01,  1.48545828e-03, -4.70589807e-03],
       [-1.48154634e-03,  9.99998554e-01,  8.34662028e-04],
       [ 4.70713112e-03, -8.27679859e-04,  9.99988579e-01]])

In [47]:
T

array([[-6.49029354e+01],
       [ 5.97272439e-01],
       [-6.36793694e-02]])